In [ ]:
# Model Training:
# 1) Load all data from preprocessing (training/test splits, etc)
# 2) Begin Training Models
    #  a) Decision Tree
    #  b) Naive Bayes
    #  c) Logistic Regression
    #  d) SVM
# 3) Testing Models
# 4) New Iterations

In [10]:
# Imports
from Models import ModelUtil
import importlib
from Data import Preprocessing
from sklearn.model_selection import train_test_split
from Logs import logging as logs

In [11]:
# 1) Load all data from preprocessing 
importlib.reload(Preprocessing)
importlib.reload(ModelUtil)
importlib.reload(logs)
infieldDataFrame, outfieldDataFrame = Preprocessing.dataFiltering()

c:\Users\Trent\Desktop\Senior Design\shifting_model\Data\DataUtil.py:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["PitcherThrows"] = df["PitcherThrows"].map({"Left":1, "Right":2, "Both":3})


In [ ]:
infieldcorrmatrix = infieldDataFrame.corr()
infieldcorrmatrix.to_csv('Infield_Correlation_Matrix')
outfieldcorrmatrix = outfieldDataFrame.corr()
outfieldcorrmatrix.to_csv('Outfield_Correlation_Matrix')

In [12]:
# 2) Training Models
Y = infieldDataFrame["FieldSlice"]
X = infieldDataFrame[["PitcherThrows", "BatterSide", "TaggedPitchType", "ZoneSpeed", "PlateLocHeight", "PlateLocSide"]]
xTrain, xTest, yTrain, yTest = train_test_split(X, Y, test_size=0.20, random_state=11)

In [13]:
# a) Decision Tree
# Need to test these hyperparameters for best case
max_depth = 25
max_features = 10
max_leaf_nodes = 55
result = ModelUtil.runDT(xTrain, yTrain, xTest, yTest, max_depth, max_features, max_leaf_nodes)


training decision tree model...
done!
getting statistics...



c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:58: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dfTestStats = dftest.groupby(["FieldSliceActual"]).size().reset_index()
c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:62: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dfTestStats["Correct"] = dftemp.groupby(["FieldSliceActual"]).size().reset_index()[0]


Model Type: DecisionTree

Training Size = 45028
Testing Size = 11257

Training Accuracy = 0.32264368837167984
Testing Accuracy = 0.31269432353202453

Hyper-Parameters: 

Max Tree Depth: 25
Max Tree Features: 10
Max Leaf Nodes: 55

Accuracy Score for Predicting on Training Data: 0.3226
Accuracy Score for Predicting on Test Data: 0.3127

Overall Average Probabilities
-------------------------------------
Section 1: 20.76%
Section 2: 20.65%
Section 3: 20.24%
Section 4: 19.54%
Section 5: 18.80%

Field Slice Counts for Training Data
--------------------------------------------------
Section	Truth	Prediction
1		7817		16175
2		11664		9772
3		11058		3585
4		9059		6395
5		5430		9101
Amount Correct: 14528
Amount Incorrect: 30500

Field Slice Counts for Testing Data
--------------------------------------------------
Section	Truth	Prediction
1		1991		4071
2		2946		2453
3		2717		904
4		2305		1555
5		1298		2274
Amount Correct: 3520
Amount Incorrect: 7737
done!


c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:64: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dfTrainStats = dftrain.groupby(["FieldSliceActual"]).size().reset_index()
c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:68: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dfTrainStats["Correct"] = dftemp.groupby(["FieldSliceActual"]).size().reset_index()[0]
c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:147: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retai

In [9]:
# b) Naive Bayes

var_smoothing = 1e-9
result = ModelUtil.runNB(xTrain, yTrain, xTest, yTest, var_smoothing)
test = True
# print("Training Accuracy: ")
# print(result[1])
# print("Testing Accuracy: ")
# print(result[2])

training Naive Bayes model...
done!
getting statistics...


c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:58: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dfTestStats = dftest.groupby(["FieldSliceActual"]).size().reset_index()
c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:62: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dfTestStats["Correct"] = dftemp.groupby(["FieldSliceActual"]).size().reset_index()[0]
c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:64: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain cu

Model Type: NaiveBayes

Training Size = 45028
Testing Size = 11257

Training Accuracy = 0.31340499244914277
Testing Accuracy = 0.31988984631784667

Hyper-Parameters: 

Var Smoothing: 1e-09

Accuracy Score for Predicting on Training Data: 0.3134
Accuracy Score for Predicting on Test Data: 0.3199

Overall Average Probabilities
-------------------------------------
Section 1: 19.31%
Section 2: 24.61%
Section 3: 23.65%
Section 4: 20.36%
Section 5: 12.07%

Field Slice Counts for Training Data
--------------------------------------------------
Section	Truth	Prediction
1		7817		8297
2		11664		21117
3		11058		472
4		9059		12094
5		5430		3048
Amount Correct: 14112
Amount Incorrect: 30916

Field Slice Counts for Testing Data
--------------------------------------------------
Section	Truth	Prediction
1		1991		2114
2		2946		5287
3		2717		97
4		2305		3027
5		1298		732
Amount Correct: 3601
Amount Incorrect: 7656


c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:147: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dfTestStats = dftest.groupby(["FieldSliceActual"]).size().reset_index()
c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:151: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dfTestStats["Correct"] = dftemp.groupby(["FieldSliceActual"]).size().reset_index()[0]
c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:153: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain

done!


In [7]:
# c)Logistic Regression
lr = 0.8
e = 100
result = ModelUtil.runLogReg(xTrain, yTrain, xTest, yTest, lr, e)
print("Training Accuracy: ")
print(result[1])
print("Testing Accuracy: ")
print(result[2])

training logistic regression model...
done!
getting statistics...
logging statistics...
Model Type: LogisticRegression

Training Size = 45028
Testing Size = 11257

Training Accuracy = 0.30105712001421336
Testing Accuracy = 0.3014124544727725

Hyper-Parameters: 

Learning Rate: 0.8
Epochs: 100

Accuracy Score for Predicting on Training Data: 0.3011
Accuracy Score for Predicting on Test Data: 0.3014

Overall Average Probabilities
-------------------------------------
Section 1: 20.66%
Section 2: 20.58%
Section 3: 20.19%
Section 4: 19.62%
Section 5: 18.96%

Field Slice Counts for Training Data
--------------------------------------------------
Section	Truth	Prediction
1		7817		15986
2		11664		10222
3		11058		3023
4		9059		3750
5		5430		12047
Amount Correct: 13556
Amount Incorrect: 31472

Field Slice Counts for Testing Data
--------------------------------------------------
Section	Truth	Prediction
1		1991		4058
2		2946		2577
3		2717		729
4		2305		897
5		1298		2996
Amount Correct: 3393
Amo

C:\Users\Trent\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:58: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dfTestStats = dftest.groupby(["FieldSliceActual"]).size().reset_index()
c:\Users

In [8]:
# d) SVM
rC = 1
kernel='linear'
degree= 1
gamma= 'scale'
coef0= 0.0
result = ModelUtil.runSVM(xTrain, yTrain, xTest, yTest, rC, kernel, degree, gamma, coef0)
print("Training Accuracy: ")
print(result[1])
print("Testing Accuracy: ")
print(result[2])

training SVM model...
done!
getting statistics...


c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:58: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dfTestStats = dftest.groupby(["FieldSliceActual"]).size().reset_index()
c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:62: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dfTestStats["Correct"] = dftemp.groupby(["FieldSliceActual"]).size().reset_index()[0]
c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:64: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain cu

Model Type: SVM

Training Size = 45028
Testing Size = 11257

Training Accuracy = 0.283867815581416
Testing Accuracy = 0.2796482188860265

Hyper-Parameters: 

Regularization Constant: 1
Kernel Type: linear
Kernel Degree1
Kernel Coefficient (gamma): scale
Independent Term in Kernel (coef0): 0.0

Accuracy Score for Predicting on Training Data: 0.2839
Accuracy Score for Predicting on Test Data: 0.2796


c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:147: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dfTestStats = dftest.groupby(["FieldSliceActual"]).size().reset_index()
c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:151: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dfTestStats["Correct"] = dftemp.groupby(["FieldSliceActual"]).size().reset_index()[0]
c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:153: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain


Overall Average Probabilities
-------------------------------------
Section 1: 17.00%
Section 2: 26.27%
Section 3: 24.53%
Section 4: 20.14%
Section 5: 12.06%

Field Slice Counts for Training Data
--------------------------------------------------
Section	Truth	Prediction
1		7817		16090.0
2		11664		13786.0
3		11058		15152.0
4		9059		nan
5		5430		nan
Amount Correct: 12782
Amount Incorrect: 32246

Field Slice Counts for Testing Data
--------------------------------------------------
Section	Truth	Prediction
1		1991		4072.0
2		2946		3426.0
3		2717		3759.0
4		2305		nan
5		1298		nan
Amount Correct: 3148
Amount Incorrect: 8109
done!
Training Accuracy: 
0.283867815581416
Testing Accuracy: 
0.2796482188860265


In [ ]:
# 3) Model Testing:

In [ ]:
# 4) Model Iterations and Improvements